# TODOs

- [x] Figure out why resumption is not working
- Verify that loss function is implemented right 
- record / plot loss better and easy background training
- [X] Research more QLoRA
  - [ ] make sure that QLoRA is actually happening..
  - RTFP
  - [X] Find good solid example code with transformers
  - Determine QLoRA parameters to experiment with and how
    - datatype
    - what bits
    - r
    - dropout
    - target_modules
    - lora alpha
- Determine training parameters to experiment with
  - AdamW RTFP
  - tune adamW starting learning rate?
  - Batch size. Should it vary over time?
  - 
- get complicated and weird
  - explore mining hard negatives
  - explore synthesizing hard negatives
  - synthesize more varied data
  - generated data off of a target domain (easy-on fine tuning)
- benchmark
  - add inferrence code path
  - figure out how to run model against an mteb benchmark
- update code to be able to run it distributed in the cloud?
  - related to deepspeed and accelerate
    - https://arxiv.org/abs/2104.07857
    - https://huggingface.co/docs/accelerate/usage_guides/deepspeed
- [x] get a bigger graphics card

In [15]:
%load_ext autoreload
%autoreload 2
seed = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import train

base_model, tokenizer = train.load_model()
model = train.load_model_for_training(base_model)

dataset = train.prepare_dataset(tokenizer, seed)
# Access the new train and test datasets
train_dataset = dataset["train"]
eval_dataset = dataset["test"]



In [14]:
# train_dataset[0]
tokenizer("foo bar baz!", add_special_tokens=True)

{'input_ids': [1, 7953, 2594, 12741, 29991, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:

from transformers import TrainingArguments, IntervalStrategy
from train import TinyEmbedTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    log_level="info",
    logging_strategy=IntervalStrategy.STEPS,
    gradient_checkpointing=True,
    logging_steps=5,
    save_steps=1000,
    seed=seed,
)

trainer = TinyEmbedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train(resume_from_checkpoint=False)

In [ ]:
from experiments import experiment_quantization
import os

experiment_quantization.run()


In [1]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers.trainer_utils import get_last_checkpoint
from train_main import load_trainer


# trainer = load_trainer(40)
# trainer._load_from_checkpoint(get_last_checkpoint(trainer.args.output_dir))
# model = trainer.model
# tokenizer = trainer.tokenizer
model = AutoModel.from_pretrained('./results_44.0/checkpoint-40000/') #'TinyLlama/TinyLlama-1.1B-intermediate-step-1195k-token-2.5T')
tokenizer = AutoTokenizer.from_pretrained('./results_44.0/checkpoint-40000/') #'TinyLlama/TinyLlama-1.1B-intermediate-step-1195k-token-2.5T') 
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import train
import pandas as pd
import torch
import torch.nn.functional as F


dataset = train.prepare_dataset(tokenizer, 42)

items = dataset["test"].select(range(5)).to_list()

results = []

for a, b in zip(items, items[1:]):
    query = a["query"]
    pos = a["pos"]
    hard_neg = a["neg"]
    soft_neg1 = b["pos"]
    soft_neg2 = b["neg"]
    embeddings = train.get_embeddings_from_text(
        model, tokenizer, [query, pos, hard_neg, soft_neg1, soft_neg2]
    )
    query_embed = embeddings[0]
    answer_embed = embeddings[1:]
    similarities = F.cosine_similarity(
        query_embed.repeat(len(answer_embed), 1), answer_embed
    )
    print("\n".join([ f"- {x}" for x in [query, pos, hard_neg, soft_neg1, soft_neg2]]))
    results.append(
        {
            "pos": similarities[0].item(),
            "hard_neg": similarities[1].item(),
            "soft_neg1": similarities[2].item(),
            "soft_neg2": similarities[3].item(),
        }
    )

df = pd.DataFrame(results)

df

- Retrieve critical reviews, analysis, and interpretations of the song 'Bohemian Rhapsody' by Queen.
- The song 'Bohemian Rhapsody' by Queen has been widely praised for its unique structure and powerful vocals. Critics have analyzed the song's operatic elements and its impact on the rock genre. Many interpretations of the lyrics have been discussed, with some suggesting it reflects the inner turmoil of the songwriter. Overall, the song has been celebrated as a masterpiece of music history.
- The song 'Bohemian Rhapsody' by Queen was released in 1975 and became a commercial success. It features a mix of rock and opera, and its lyrics have been the subject of much speculation. The song's popularity has led to numerous covers and references in popular culture. It remains a beloved classic and a staple of Queen's discography.
- If your smartphone screen is frozen, there are several troubleshooting steps you can take to resolve the issue. First, try restarting the device by holding down the

,pos,hard_neg,soft_neg1,soft_neg2
0,0.736525,-0.141566,0.113576,-0.275082
1,0.889012,-0.187596,0.008523,-0.197920
2,0.833286,-0.211161,-0.007408,-0.217319
3,0.896245,-0.234015,-0.021853,-0.175640


In [4]:
from train import get_embeddings_from_text 

items = [
  "There was a car crash on I-94. Two semi-trucks collided",
  "I like waffles",
  "I like hot dogs",
  "I like french fries",
  "The corporate lunch was awful",
  "What happened in the news yesterday?",
]
embeddings = get_embeddings_from_text(model, tokenizer, items)

data = []
for text_a, embedding_a in zip(items, embeddings):
  row = { "text": text_a }
  data.append(row)
  for text_b, embedding_b in zip(items, embeddings):
    row[text_b] = F.cosine_similarity(embedding_a, embedding_b, dim=0).item()
df = pd.DataFrame(data=data)
df

,text,There was a car crash on I-94. Two semi-trucks collided,I like waffles,I like hot dogs,I like french fries,The corporate lunch was awful,What happened in the news yesterday?
0,There was a car crash on I-94. Two semi-trucks...,1.000000,0.078175,0.214723,0.248861,0.076797,0.393608
1,I like waffles,0.078175,1.000000,0.217257,0.334502,0.051545,0.107020
2,I like hot dogs,0.214723,0.217257,1.000000,0.452492,0.166939,0.225391
3,I like french fries,0.248861,0.334502,0.452492,1.000000,0.192974,0.184606
4,The corporate lunch was awful,0.076797,0.051545,0.166939,0.192974,1.000000,0.040765
5,What happened in the news yesterday?,0.393608,0.107020,0.225391,0.184606,0.040765,1.000000


In [17]:
%load_ext autoreload
%autoreload 2
from train_main import load_trainer

from transformers.trainer_utils import get_last_checkpoint
from train_main import load_trainer


trainer = load_trainer(40)
trainer._load_from_checkpoint(get_last_checkpoint(trainer.args.output_dir))
model = trainer.model
tokenizer = trainer.tokenizer



loading configuration file config.json from cache at /mnt/freezer/huggingface-cache/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1195k-token-2.5T/snapshots/706bc2851338c4a89bb212e96ff23f9cc1ebde1d/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file model.safetensors from cache at /mnt/freezer/huggingface-cache/hub/models--TinyLlama--TinyLlama-1.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of the model checkpoint at TinyLlama/TinyLlama-1.1B-intermediate-step-1195k-token-2.5T were not used when initializing LlamaModel: ['lm_head.weight']
- This IS expected if you are initializing LlamaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of LlamaModel were initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-intermediate-step-1195k-token-2.5T.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaModel for predictions without further training.
loading file tokenizer.model from cache at /mnt/freezer/huggingface-cache/hub/models--TinyLl

In [18]:
import torch
import train
from datasets import Dataset
import pandas as pd
dataset: Dataset = trainer.eval_dataset
batch = dataset.select(range(2)).to_list()



In [19]:
collated = trainer._map_collate_fn(batch)

# the embeddings here seem normal size
trainer.compute_loss(model, collated)

results = []

for a, b in zip(batch, batch[1:]):
    query = a["query"]
    pos = a["pos"]
    hard_neg = a["neg"]
    soft_neg1 = b["pos"]
    soft_neg2 = b["neg"]
    embeddings = train.get_embeddings_from_text(
        model, tokenizer, [query, pos, hard_neg, soft_neg1, soft_neg2]
    )
    # these embeddings are way to small. Close to 0 (negative and positive)
    query_embed = embeddings[0]
    answer_embed = embeddings[1:]
    similarities = torch.nn.functional.cosine_similarity(
        query_embed.repeat(len(answer_embed), 1), answer_embed
    )
    print("\n".join([ f"- {x}" for x in [query, pos, hard_neg, soft_neg1, soft_neg2]]))
    results.append(
        {
            "pos": similarities[0].item(),
            "hard_neg": similarities[1].item(),
            "soft_neg1": similarities[2].item(),
            "soft_neg2": similarities[3].item(),
        }
    )

df = pd.DataFrame(results)

df




query1 tensor([ 0.0062,  0.0072,  0.0361,  ..., -0.1122, -0.0163,  0.0068],
       grad_fn=<SelectBackward0>)
query1 input_ids tensor([    1,  4649, 29878,  2418, 12187, 21804, 29892,  7418, 29892,   322,
         6613,   800,   310,   278,  4823,   525, 29933,  1148,   331,   713,
          390,  4252,  1486, 29915,   491, 10470, 29889,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
{'pos': 0.6921306848526001, 'neg': -0.01485941807

,pos,hard_neg,soft_neg1,soft_neg2
0,0.997785,0.997139,0.986794,0.992242


In [6]:
trainer.compute_loss(trainer.model, batch)


all_embeddings tensor([[ 0.0261, -0.0123,  0.0158,  ...,  0.0009, -0.0283,  0.0222],
        [ 0.0133,  0.0144, -0.0191,  ...,  0.0392, -0.0007,  0.0102],
        [-0.0087, -0.0292, -0.0002,  ...,  0.0412, -0.0095,  0.0213],
        [ 0.0299, -0.0183, -0.0254,  ...,  0.0640, -0.0162,  0.0198]],
       grad_fn=<DivBackward0>)
{'pos': 0.6921306848526001, 'neg': -0.014859418074289955}


0.6921306848526001